<a href="https://colab.research.google.com/github/yaPhilya/Computer-Vision/blob/master/hw4/colab_trainning_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
!pip install keras

In [3]:
import keras
import keras.layers as L

Using TensorFlow backend.


In [4]:
from google.colab import drive
drive.mount('/content/hw4')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/hw4


In [5]:
import os
os.listdir('/content/hw4/My Drive/Colab Notebooks/Computer Vision/hw4_2')

['stupid_model_2.hdf5',
 'stupid_model.hdf5',
 'gt.csv',
 'img_shapes.csv',
 'train']

In [0]:
PATH = '/content/hw4/My Drive/Colab Notebooks/Computer Vision/hw4_2/'

In [7]:
points_data = pd.read_csv(PATH + "gt.csv")
points_data.head()

,filename,x1,y1,x2,y2,x3,y3,x4,y4,x5,...,x10,y10,x11,y11,x12,y12,x13,y13,x14,y14
0,00000.jpg,51,51,127,60,169,65,203,52,68,...,193,84,156,144,92,174,140,180,168,179
1,00001.jpg,21,20,36,18,57,19,80,25,23,...,78,33,43,46,31,61,45,66,60,65
2,00002.jpg,23,116,39,83,47,57,56,19,36,...,62,31,91,86,106,122,115,100,121,81
3,00003.jpg,29,18,63,25,84,32,105,34,38,...,95,45,72,71,46,83,63,87,75,89
4,00004.jpg,31,53,76,44,107,38,136,33,40,...,129,49,111,105,72,137,106,132,126,121


In [8]:
size_data = pd.read_csv(PATH + "img_shapes.csv")
size_data.head()

,img_filename,n_rows,n_cols
0,00000.jpg,262,262
1,00001.jpg,94,94
2,00002.jpg,166,166
3,00003.jpg,124,124
4,00004.jpg,189,189


In [0]:
from skimage import io
from skimage.transform import resize

In [0]:
from tqdm import tqdm

In [11]:
images = []
shapes = []
NEW_SHAPE = 100
for row in tqdm(size_data.iterrows()):
    im = io.imread(PATH + "train/images/{}".format(row[1]['img_filename']))
    images.append(resize(im, (NEW_SHAPE, NEW_SHAPE, 3)))
    shapes.append((row[1]['n_rows'], row[1]['n_cols']))
images = np.array(images)
shapes = np.array(shapes)

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
6000it [26:46,  4.07it/s]


In [12]:
points = []
for i in range(shapes.shape[0]):
    point = []
    for col in points_data.columns:
        if col.startswith('y'):
            point.append(float(points_data.iloc[i][col]) / float(shapes[i][0]) * float(NEW_SHAPE))
        if col.startswith('x'):
            point.append(float(points_data.iloc[i][col]) / float(shapes[i][1]) * float(NEW_SHAPE))
    points.append(point)
points = np.array(points, dtype=np.float64)
points.shape

(6000, 28)

In [13]:
points[:5]

array([[19.46564885, 19.46564885, 48.47328244, 22.90076336, 64.50381679,
        24.80916031, 77.48091603, 19.84732824, 25.95419847, 29.38931298,
        32.44274809, 28.6259542 , 41.98473282, 31.29770992, 61.06870229,
        33.20610687, 65.26717557, 30.15267176, 73.66412214, 32.0610687 ,
        59.54198473, 54.96183206, 35.11450382, 66.41221374, 53.4351145 ,
        68.70229008, 64.1221374 , 68.32061069],
       [22.34042553, 21.27659574, 38.29787234, 19.14893617, 60.63829787,
        20.21276596, 85.10638298, 26.59574468, 24.46808511, 28.72340426,
        36.17021277, 27.65957447, 42.55319149, 29.78723404, 62.76595745,
        31.91489362, 72.34042553, 31.91489362, 82.9787234 , 35.10638298,
        45.74468085, 48.93617021, 32.9787234 , 64.89361702, 47.87234043,
        70.21276596, 63.82978723, 69.14893617],
       [13.85542169, 69.87951807, 23.4939759 , 50.        , 28.31325301,
        34.3373494 , 33.73493976, 11.44578313, 21.68674699, 69.27710843,
        24.09638554, 57.8313

In [20]:
model = keras.models.Sequential()
model.add(L.Convolution2D(filters=16, kernel_size=3, input_shape=(NEW_SHAPE, NEW_SHAPE, 3)))
model.add(L.BatchNormalization())
model.add(L.Activation('relu'))
model.add(L.Convolution2D(filters=32, kernel_size=3))
model.add(L.BatchNormalization())
model.add(L.Activation('relu'))
model.add(L.MaxPooling2D())

model.add(L.Convolution2D(filters=64, kernel_size=3))
model.add(L.BatchNormalization())
model.add(L.Activation('relu'))
model.add(L.Convolution2D(filters=128, kernel_size=3))
model.add(L.BatchNormalization())
model.add(L.Activation('relu'))
model.add(L.MaxPooling2D())

model.add(L.Convolution2D(filters=256, kernel_size=3))
model.add(L.BatchNormalization())
model.add(L.Activation('relu'))
model.add(L.Convolution2D(filters=512, kernel_size=3))
model.add(L.BatchNormalization())
model.add(L.Activation('relu'))
model.add(L.MaxPooling2D())

model.add(L.Flatten())
model.add(L.Dense(units=128, activation='relu'))
model.add(L.Dense(units=64, activation='relu'))
model.add(L.Dense(units=28))
model.compile('adam', loss='mse')
model.build((None, NEW_SHAPE, NEW_SHAPE, 3))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 98, 98, 16)        448       
_________________________________________________________________
batch_normalization_7 (Batch (None, 98, 98, 16)        64        
_________________________________________________________________
activation_7 (Activation)    (None, 98, 98, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 96, 96, 32)        4640      
_________________________________________________________________
batch_normalization_8 (Batch (None, 96, 96, 32)        128       
_________________________________________________________________
activation_8 (Activation)    (None, 96, 96, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 48, 48, 32)        0         
__________

In [0]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test, shape_train, shape_test = train_test_split(images, points, shapes, test_size=0.1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape, shape_train.shape, shape_test.shape

((5400, 100, 100, 3),
 (600, 100, 100, 3),
 (5400, 28),
 (600, 28),
 (5400, 2),
 (600, 2))

In [0]:
model.compile(keras.optimizers.Adam(), loss='mse')

In [22]:
model.fit(X_train, y_train, batch_size=512, epochs=120, verbose=1, validation_data=(X_test, y_test))

Train on 5400 samples, validate on 600 samples
Epoch 1/120
5400/5400 [==============================] - 16s 3ms/step - loss: 906.4968 - val_loss: 746.0026
Epoch 2/120
5400/5400 [==============================] - 14s 3ms/step - loss: 79.9755 - val_loss: 6981.2945
Epoch 3/120
5400/5400 [==============================] - 14s 3ms/step - loss: 49.3604 - val_loss: 2067.8841
Epoch 4/120
5400/5400 [==============================] - 14s 3ms/step - loss: 29.9928 - val_loss: 269.9079
Epoch 5/120
5400/5400 [==============================] - 14s 3ms/step - loss: 26.5375 - val_loss: 87.5270
Epoch 6/120
5400/5400 [==============================] - 14s 3ms/step - loss: 24.0723 - val_loss: 40.6648
Epoch 7/120
5400/5400 [==============================] - 14s 3ms/step - loss: 22.9692 - val_loss: 32.0953
Epoch 8/120
5400/5400 [==============================] - 14s 3ms/step - loss: 22.1295 - val_loss: 27.9786
Epoch 9/120
5400/5400 [==============================] - 14s 3ms/step - loss: 21.5221 - val_loss: 

In [19]:
model.fit(X_train, y_train, batch_size=512, epochs=240, verbose=1, validation_data=(X_test, y_test), initial_epoch=120)

Train on 5400 samples, validate on 600 samples
Epoch 121/240
5400/5400 [==============================] - 14s 3ms/step - loss: 2.9775 - val_loss: 12.6330
Epoch 122/240
5400/5400 [==============================] - 14s 3ms/step - loss: 3.0225 - val_loss: 12.6021
Epoch 123/240
5400/5400 [==============================] - 14s 3ms/step - loss: 2.9944 - val_loss: 13.5605
Epoch 124/240
5400/5400 [==============================] - 14s 3ms/step - loss: 3.0323 - val_loss: 12.3422
Epoch 125/240
5400/5400 [==============================] - 14s 3ms/step - loss: 2.9250 - val_loss: 12.3138
Epoch 126/240
5400/5400 [==============================] - 14s 3ms/step - loss: 2.9259 - val_loss: 12.4471
Epoch 127/240
5400/5400 [==============================] - 14s 3ms/step - loss: 2.8935 - val_loss: 12.5576
Epoch 128/240
5400/5400 [==============================] - 14s 3ms/step - loss: 2.8427 - val_loss: 12.4503
Epoch 129/240
5400/5400 [==============================] - 14s 3ms/step - loss: 2.9897 - val_loss

KeyboardInterrupt: ignored

In [0]:
model.compile(keras.optimizers.Adam(), loss='mse')

In [95]:
X_train, X_test, y_train, y_test, shape_train, shape_test = train_test_split(images, points, shapes, test_size=0.1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape, shape_train.shape, shape_test.shape

((5400, 100, 100, 3),
 (600, 100, 100, 3),
 (5400, 28),
 (600, 28),
 (5400, 2),
 (600, 2))

In [96]:
model.fit(X_train, y_train, batch_size=256, epochs=98, verbose=1, validation_data=(X_test, y_test))

Train on 5400 samples, validate on 600 samples
Epoch 1/98
5400/5400 [==============================] - 24s 4ms/step - loss: 37.0524 - val_loss: 694.3705
Epoch 2/98
5400/5400 [==============================] - 16s 3ms/step - loss: 8.1859 - val_loss: 35.6170
Epoch 3/98
5400/5400 [==============================] - 17s 3ms/step - loss: 4.9971 - val_loss: 11.0100
Epoch 4/98
5400/5400 [==============================] - 16s 3ms/step - loss: 3.8979 - val_loss: 14.6874
Epoch 5/98
5400/5400 [==============================] - 16s 3ms/step - loss: 2.9919 - val_loss: 26.6359
Epoch 6/98
5400/5400 [==============================] - 16s 3ms/step - loss: 2.7654 - val_loss: 14.1308
Epoch 7/98
5400/5400 [==============================] - 16s 3ms/step - loss: 2.2250 - val_loss: 9.4205
Epoch 8/98
5400/5400 [==============================] - 16s 3ms/step - loss: 2.0815 - val_loss: 8.8571
Epoch 9/98
5400/5400 [==============================] - 16s 3ms/step - loss: 2.2173 - val_loss: 7.5156
Epoch 10/98
5400/5

KeyboardInterrupt: ignored

In [0]:
model.save(PATH + 'stupid_model_2.hdf5')